<img src="images/JHI_STRAP_Web.png" style="width: 150px; float: right;">
# 03b - An Example Reproducible Document

## Table of Contents

1. [Introduction](#introduction)
2. [Python Imports/Startup](#python_imports)
3. [Biological Motivation](#motivation)
4. [Load Sequence](#load_sequence)
5. [Build `BLAST` database](#build_blast)
6. [Run `BLAST` query](#blast_query)
7. [Load `BLAST` results](#blast_results)

<a id="introduction"></a>
## 1. Introduction

<p></p><div class="alert-success">
<b>This notebook is an example of what could be done in part `03b - Building a Reproducible Document` if more time was available.</b>
</div>

It is provided as a reference against which you can check your work from notebook `03b`, and inspect to get ideas for how you can use the same techniques and methods in your own research.

<a id="python_imports"></a>
## 2. Python Imports/Startup

<p></p><div class="alert-success">
<b>It can be very convenient to have all the `Python` library imports at the top of the notebook.</b>
</div>

This is very helpful when running the notebook with, e.g. `Cell -> Run All` or `Kernel -> Restart & Run All` from the menu bar, all the libraries are available throughout the document.

In [1]:
# The line below allows the notebooks to show graphics inline
%pylab inline

import os                            # This lets us communicate with the operating system

import pandas as pd                  # This lets us use dataframes
import seaborn as sns                # This lets us draw pretty graphics

# Biopython is a widely-used library for bioinformatics
# tasks, and integrating with software
from Bio import SeqIO                # This lets us handle sequence data
from Bio.KEGG import REST            # This lets us connect to the KEGG databases

# The bioservices library allows connections to common
# online bioinformatics resources
from bioservices import UniProt      # This lets us connect to the UniProt databases

from IPython.display import Image    # This lets us display images (.png etc) from code

Populating the interactive namespace from numpy and matplotlib


<p></p><div class="alert-success">
<b>It can be useful here to create any output directories that will be used throughout the document.</b>
</div>

The `os.makedirs()` function allows us to create a new directory, and the `exist_ok` option will prevent the notebook code from stopping and throwing an error if the directory already exists.

In [2]:
# Create a new directory for notebook output
OUTDIR = os.path.join("data", "reproducible", "output")
os.makedirs(OUTDIR, exist_ok=True)

<a id="motivation"></a>
## 3. Biological Motivation

<p></p><div class="alert-info">
<b>We are working on a project to improve bacterial throughput for biosynthesis, and have been provided with a nucleotide sequence of a gene of interest.
<br></br><br></br>
This gene is overrepresented in populations of bacteria that appear to be associated with enhanced metabolic function relevant to a biosynthetic output (lipid conversion to ethanol).
<br></br><br></br>
We want to find out more about the annotated function and literature associated with this gene, which appears to derive from *Proteus mirabilis*.
</div>

Our plan is to:

1. identify a homologue in a reference isolate of *P. mirabilis*
2. obtain the protein sequence/identifier for the homologue
3. get information about the molecular function of this protein from `UniProt`
4. get information about the metabolic function of this protein from `KEGG`
5. visualise some of the information about this gene/protein

<a id="load_sequence"></a>
## 4. Load Sequence

<p></p><div class="alert-success">
<b>We first load the sequence from a local `FASTA` file, using the `Biopython` `SeqIO` library.</b>
</div>

In the `code` cell below:

* we put the path to the sequence file in the variable `seqfile`
* we load the sequence into the variable `wildtype`

In [3]:
# Define the path to the sequence file (in data/reproducible/sequences)
seqfile = os.path.join("data", "reproducible", "sequences", "lipase.fasta")

# Load the sequence data
wildtype = SeqIO.read(seqfile, 'fasta')

Once the sequence is loaded, we print it to the notebook to check it is correct.

In [4]:
# Print the 'wildtype' sequence
print(wildtype)

ID: candidate
Name: candidate
Description: candidate lipase protein from Proteus mirabilis population
Number of features: 0
Seq('ATGAGCACCAAGTACCCCATCGTGCTGGTGCACGGCCTGGCCGGCTTCAGCGAG...CTG', SingleLetterAlphabet())


To see the sequence in `FASTA` format, we can use the `.format()` method:

In [10]:
# Print the 'wildtype' sequence as a FASTA formatted sequence
print(wildtype.format('fasta'))
print(len(wildtype))

>candidate lipase protein from Proteus mirabilis population
ATGAGCACCAAGTACCCCATCGTGCTGGTGCACGGCCTGGCCGGCTTCAGCGAGATCGTG
GGCTTCCCCTACTTCTACGGCATCGCCGACGCCCTGACCCAGGACGGCCACCAGGTGTTC
ACCGCCAGCCTGAGCGCCTTCAACAGCAACGAGGTGAGGGGCAAGCAGCTGTGGCAGTTC
GTGCAGACCATCCTGCAGGAGACCCAGACCAAGAAGGTGAACTTCATCGGCCACAGCCAG
GGCCCCCTGGCCTGCAGGTACGTGGCCGCCAACTACCCCGACAGCGTGGCCAGCGTGACC
AGCATCAACGGCGTGAACCACGGCAGCGAGATCGCCGACCTGTACAGGAGGATCATCAGG
AAGGACAGCATCCCCGAGTACATCGTGGAGAAGGTGCTGAACGCCTTCGGCACCATCATC
AGCACCTTCAGCGGCCACAGGGGCGACCCCCAGGACGCCATCGCCGCCCTGGAGAGCCTG
ACCACCGAGCAGGTGACCGAGTTCAACAACAAGTACCCCCAGGCCCTGCCCAAGACCCCC
TGCGGCGAGGGCGACGAGATCGTGAACGGCGTGCACTACTACTGCTTCGGCAGCTACATC
CAGGAGCTGATCGCCGGCGAGAACGGCAACCTGCTGGACCCCACCCACGCCGCCATGAGG
GTGCTGAACACCCTGTTCACCGAGAAGCAGAACGACGGCCTGGTGGGCAGGTGCAGCATG
AGGCTGGGCAAGCTGATCAAGGACGACTACGCCCAGGACCACTTCGACATGGTGAACCAG
GTGGCCGGCCTGGTGAGCTACAACGAGAACATCGTGGCCATCTACACCCTGCACGCCAAG
TACCTGGCCAGCAAGCAGCTG

861


<a id="build_blast"></a>
## 5. Build `BLAST` Database

<p></p><div class="alert-success">
<b>We now build a local `BLAST` database from the *P. mirabilis* reference proteins.</b>
</div>

* to do this, we run terminal commands in the notebook by using the "*magic*" command `%%bash` at the top of the `code` cell
* using the `%%bash` command makes the rest of the lines in the cell work as though they were the command-line terminal

We build a `BLAST` protein database from the file `data/reproducible/sequences/GCA_000069965.1_ASM6996v1_protein.faa` using the `makeblastdb` command.

<p></p><div class="alert-info">
The output from this command is printed in the notebook below the cell when it runs.
</div>

In [6]:
%%bash
# Drop out to bash to create the BLAST database

# Make a BLAST protein database
makeblastdb -in data/reproducible/sequences/GCA_000069965.1_ASM6996v1_protein.faa \
            -dbtype prot \
            -title reference_protein \
            -out data/reproducible/output/reference_protein



Building a new DB, current time: 04/16/2017 12:44:26
New DB name:   /Users/lpritc/Development/GitHub/Teaching-SfAM-ECR/workshop/data/reproducible/output/reference_protein
New DB title:  reference_protein
Sequence type: Protein
Deleted existing Protein BLAST database named /Users/lpritc/Development/GitHub/Teaching-SfAM-ECR/workshop/data/reproducible/output/reference_protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 3662 sequences in 0.15228 seconds.


<a id="blast_query"></a>
## 6. Run `BLAST` Query

<p></p><div class="alert-success">
<b>We now query the wildtype sequence against our custom `BLAST` database from the *P. mirabilis* reference proteins.</b>
</div>

* to do this, we again run terminal commands in the notebook by using the "*magic*" command `%%bash` at the top of the `code` cell

We `BLASTX` the nucleotide sequence query against the `BLAST` protein database in `data/reproducible/output/reference_protein`.

<p></p><div class="alert-info">
We set the `BLAST` output format to `6` (tabular) format, so we can read it in with `pandas`.
<br></br><br></br>
<b>The `BLASTX` command does not produce output in the notebook, if it is successful</b>
<br></br><br></br>
We have named the `BLASTX` output file in such a way that we can tell what the query was, what the database was, and which `BLAST` program was used. This makes it easier to identify results when looking through output files.
</div>

In [7]:
%%bash
# Drop out to bash to run the BLASTX query

blastx -query data/reproducible/sequences/lipase.fasta \
        -db data/reproducible/output/reference_protein \
        -outfmt 6 \
        -out data/reproducible/output/wildtype_blastx_reference_protein.tab

<a id="blast_results"></a>
## 6. Load `BLAST` Results

<p></p><div class="alert-success">
<b>We now load the `BLASTX` results for inspection and visualisation, using `pandas`</b>
</div>

* we use the `pandas` `read_csv()` function to load the tabular data that was created using the option `-outfmt 6` in the `blastx` command
* we define column headers, to make the tabular results easier to understand

<p></p><div class="alert-info">
The tabular format is tab-separated, so we need to specify `sep="\t"` as the separator.
<br></br><br></br>
The tabular output has no header row, so we need to specify `header=None`
<br></br><br></br>
We add column headers to the dataframe *after* it is loaded
</div>

In [8]:
# Define the path to the output file
blastfile = os.path.join("data", "reproducible", "output",
                         "wildtype_blastx_reference_protein.tab")

# Define column headers
# These are the default columns produced with -outfmt 6
headers = ['query', 'subject',
           'pc_identity', 'aln_length',
           'mismatches', 'gaps_opened',
           'query_start', 'query_end',
           'subject_start', 'subject_end',
           'e_value', 'bitscore']

# Load the BLAST output into a dataframe
results = pd.read_csv(blastfile, sep="\t", header=None)
results.columns = headers

We can now inspect the first few rows of the dataframe, to see what the best hits were in the reference genome:

In [9]:
# Show first few rows of the dataframe
results.head()

,query,subject,pc_identity,aln_length,mismatches,gaps_opened,query_start,query_end,subject_start,subject_end,e_value,bitscore
0,candidate,CAR42190.1,95.122,287,14,0,1,861,1,287,0.00,567.0
1,candidate,CAR41557.1,28.947,76,44,3,598,795,93,168,0.59,28.1
2,candidate,CAR45487.1,34.043,47,31,0,235,375,131,177,0.78,27.7
3,candidate,CAR42262.1,43.478,23,13,0,727,795,191,213,1.60,26.9
4,candidate,CAR42804.1,32.258,31,21,0,721,813,33,63,3.10,24.3


There is a single best match - `CAR42190.1` - that aligns over the full length of the candidate sequence (`861/3 = 287`) with 95% identity and 14 residue mismatches.

It is very likely that this protein shares the same molecular and metabolic function as our candidate sequence, so we proceed to query the `UniProt` and `KEGG` databases.